In [87]:
from numpy import array
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import re

file_content = open(r"Database\bread.txt").read()
file_content = file_content.replace(".", " . ")
file_content = file_content.replace("\n", " \n ")

# Use regex to take off some meta-lines
file_content = re.sub(r'^   NYC.*\n?', '', file_content, flags=re.MULTILINE)
file_content = re.sub(r'^ ====.*\n?', '', file_content, flags=re.MULTILINE)
file_content = re.sub(r'^ \*\*.*\n?', '', file_content, flags=re.MULTILINE)



lines = file_content.split("@@@@@ Now You're Cooking! Export Format")
print(len(lines))
print(lines[:4])

998
['', ' \n  \n 100% Whole Wheat Bread \n  \n breads \n  \n 2/3 cup water \n 1 tablespoon sugar \n 2/3 cup shortening \n 1/2 cup molasses \n 12 cup whole wheat flour \n 3 pkg yeast \n 8 cup scalded milk \n 1 cup sugar \n 2 tablespoon salt \n  \n Dissolve yeast in 2/3 c water while your milk is cooling .   Dissolve 1 \n cup sugar in the hot milk .   Stir all ingredients in large bowl, turn \n out and knead about 5 minutes, adding flour if needed .   Knead about 5 \n minutes .  Let rise until doubled in bulk, about 1 1/2 to 2 hours .  \n Knead down and shape into 6 loaves, let rise until doubled in pans .  \n Bake at 375 degrees F .  for 40 minutes .   Turn out on wire rack and let \n cool to cold before slicing, if you can .  NOTE: Raisins and/or walnuts \n can be added for a change .  Also this bread freezes well .  \n  \n Yield: 6 servings \n  \n  \n  \n ', " \n  \n Abadoo's Banana Bread \n  \n breads \n  \n 1 3/4 cup all-purpose flour \n 2 teaspoon baking powder \n 1/4 teaspoon sod

Agora utilizamos `Tokenizer()` para gerar uma lista de palavras, onde cada palavra é um id. Lembrando que precisamos fazer uma outra lista com todas as combinações das formações das frases, já que queremos prever a próxima palavra com base nas N palavras anteriores

In [88]:
tokenizer = Tokenizer(num_words=1000, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t')


def get_sequence_of_tokens(corpus):
    ## tokenization
    janela = 100
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            if(i<janela):
                n_gram_sequence = token_list[:i+1]
                input_sequences.append(n_gram_sequence)
            else:
                n_gram_sequence = token_list[i+1-janela:i+1]
                input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(lines)
inp_sequences[:10]

[[1, 1],
 [1, 1, 910],
 [1, 1, 910, 166],
 [1, 1, 910, 166, 160],
 [1, 1, 910, 166, 160, 33],
 [1, 1, 910, 166, 160, 33, 1],
 [1, 1, 910, 166, 160, 33, 1, 1],
 [1, 1, 910, 166, 160, 33, 1, 1, 32],
 [1, 1, 910, 166, 160, 33, 1, 1, 32, 1],
 [1, 1, 910, 166, 160, 33, 1, 1, 32, 1, 1]]

In [89]:
print("word_index : ",tokenizer.word_index)
print(len(tokenizer.word_index))

word_index :  {'\n': 1, '.': 2, '1': 3, '2': 4, 'and': 5, 'the': 6, 'cup': 7, 'in': 8, 'a': 9, 'to': 10, 'flour': 11, '4': 12, 'teaspoon': 13, '3': 14, 'with': 15, 'until': 16, 'or': 17, 'of': 18, 'minutes': 19, 'sugar': 20, 'for': 21, 'dough': 22, 'into': 23, 'baking': 24, 'salt': 25, 'on': 26, 'add': 27, 'tablespoon': 28, 'water': 29, 'butter': 30, 'yield': 31, 'breads': 32, 'bread': 33, 'bowl': 34, 'milk': 35, 'servings': 36, 'bake': 37, 'about': 38, 'egg': 39, 'place': 40, 'mix': 41, 'stir': 42, 'oven': 43, 'powder': 44, 'mixture': 45, 'let': 46, 'warm': 47, 'at': 48, 'yeast': 49, 'pan': 50, 'well': 51, 'oil': 52, 'from': 53, 'eggs': 54, 'it': 55, 'each': 56, 'dry': 57, 'is': 58, 'all': 59, '10': 60, 'loaf': 61, 'by': 62, '8': 63, 'out': 64, 'cool': 65, 'cover': 66, 'large': 67, 'lightly': 68, 'combine': 69, 'ingredients': 70, 'beat': 71, 'cheese': 72, 'over': 73, 'inch': 74, 'cut': 75, '5': 76, 'rise': 77, '6': 78, 'brown': 79, 'together': 80, 'soda': 81, 'greased': 82, 'heat': 83

### Ta vendo que ali em cima os vetores tem dimensões todas diferentes? precisamos padronizar isso daí fazendo com que todos tenham as mesmas dimensões

In [90]:
max_sequence_len = max([len(x) for x in inp_sequences])
input_sequences = np.array(pad_sequences(inp_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   1,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

### O último número é sempre função dos números anteriores

In [91]:
qtd_de_palavrinhas = 1000#len(tokenizer.word_index)+1
print(qtd_de_palavrinhas)

1000


In [92]:
X, y_denso = input_sequences[:,:-1],input_sequences[:,-1]
y = ku.to_categorical(y_denso, num_classes=qtd_de_palavrinhas)
print(X[:10]) # Entrada
print(y_denso[:10]) # Saída
print(y[:10]) # Saída

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   1   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0 

In [93]:
tamanho_entrada = X.shape[1]
print(tamanho_entrada)

99


O layer `Embbedded` é usado para transformar cada palavra (que agora é um número) em uma representação vetorial de acordo com sua frequência. A ideia é que palavras que sejam usadas

> words that have similar context will have similar meanings

Esse embedding é bonitão pq já arruma a matriz no formato 3D que o LSTM precisa como input, então não preciso me procupar com isso.

After we’ve done the file reading, we will create the actual input for the Network. We’re gonna use Keras to create and train our Network, so we must convert the data into this form: **(number_of_sequences, length_of_sequence, number_of_features)**.

In [94]:
model = Sequential()
model.add(Embedding(qtd_de_palavrinhas, round(qtd_de_palavrinhas**0.25), input_length=tamanho_entrada))
model.add(LSTM(100))
model.add(Dense(qtd_de_palavrinhas, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 99, 6)             6000      
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               42800     
_________________________________________________________________
dense_7 (Dense)              (None, 1000)              101000    
Total params: 149,800
Trainable params: 149,800
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.fit(X, y, epochs=20, verbose=5, batch_size=300)

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [96]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [100]:
print(generate_text("Scald the milk in a 1-quart saucepan",100,model,max_sequence_len))

Scald The Milk In A 1-Quart Saucepan 
 
 
 
 1 Cup Butter 
 1 Cup Sugar 
 1 Teaspoon Baking Powder 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon Salt 
 1 2 Teaspoon
